In [ ]:
from model.alexnet import Alexnet_for_CAM

In [ ]:
model = Alexnet_for_CAM()

In [ ]:
import pretrainedmodels

In [ ]:
print(pretrainedmodels.model_names)

In [ ]:
print(pretrainedmodels.pretrained_settings['inceptionv3'])

In [ ]:
model_name = 'inceptionv4'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

In [ ]:
print(model)

In [ ]:
from torchvision.models import googlenet, vgg16, vgg19 

In [ ]:
model = googlenet(pretrained=True)
# model = vgg16(pretrained=True)

In [ ]:
print(model)

- models that include dropout
    - GoogleNet(InceptionV1), AlexNet, VGG19

## Classification Test

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='2'

In [ ]:
from tqdm import tqdm, tqdm_notebook

import torch
import torch.nn as nn
from torchvision.models import vgg19, vgg16_bn

import dataloader

In [ ]:
valid_loader = dataloader.imagenet_loader(bs=32)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = vgg16_bn(pretrained=True)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
def test_accuracy(epoch):
    top1_accuracy = 0.
    top5_accuracy = 0.
    loss = 0.

    model.eval()
    for idx, (inputs, targets) in tqdm_notebook(enumerate(valid_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        loss += criterion(outputs, targets).detach().cpu().item()

        topk = outputs.topk(5,dim=1)[1]
        top1_accuracy += topk[:,0].eq(targets).sum().cpu().item()
        top5_accuracy += topk.eq(torch.stack([targets]*5,dim=1)).max(1)[0].sum().cpu().item()
    
    top1_accuracy /= len(valid_loader.dataset)
    top5_accuracy /= len(valid_loader.dataset)
    loss /= len(valid_loader.dataset)

    print('Classification')
    print(f'===> Test Loss: {loss:.4f}, Top1-Acc: {top1_accuracy*100:.4f}, Top5-Acc: {top5_accuracy*100:.4f}')

In [ ]:
test_accuracy(0) 
# vgg19 ===> Test Loss: 0.0349, Top1-Acc: 72.3760, Top5-Acc: 90.8760
# vgg16_bn ===> Test Loss: 0.0333, Top1-Acc: 73.3600, Top5-Acc: 91.5160

## Localization Test

In [ ]:
import json

## Classification & Localization (top-1 Loc)

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [ ]:
from tqdm import tqdm, tqdm_notebook

import torch
import torch.nn as nn
from torchvision.models import vgg16, vgg16_bn

import dataloader

In [ ]:
import util

In [ ]:
import csv
from os.path import join,expanduser
import re
import numpy as np
from PIL import Image

In [ ]:
from cam import CAM

In [ ]:
valid_loader = dataloader.imagenet_loader(bs=10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = vgg16_bn(pretrained=True)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
map = CAM()
data_dict = map.valid_dataset.data_dict
input_files = map.valid_dataset.img_files
img_dir = map.valid_dataset.img_dir

In [ ]:
bboxes_dict = {}
with open('runs/vgg16bn/0.2/bbox.csv','r') as cf:
    for row in csv.reader(cf):
        k,v = row
        bboxes_dict[k] = eval(v)

In [ ]:
len(bboxes_dict)

In [ ]:
### def test_accuracy(epoch):

top1_acc_cls = 0.
top5_acc_cls = 0.
gtknown_acc_loc = 0.
top1_acc_loc = 0.
top5_acc_loc = 0.
loss = 0.

count = 0
model.eval()
for idx, (inputs, targets) in tqdm_notebook(enumerate(valid_loader)):
    inputs, targets = inputs.to(device), targets.to(device)
    
    outputs = model(inputs)
    loss += criterion(outputs, targets).detach().cpu().item()
    
    topk = outputs.topk(5,dim=1)[1]
    top1_acc_clss = topk[:,0].eq(targets).cpu().numpy()
    top5_acc_clss = topk.eq(torch.stack([targets]*5,dim=1)).max(1)[0].cpu().numpy()
    
    gtknown_acc_locs = []
    for data_idx in range(idx*10,(idx+1)*10):
        count += 1
    
        # get true bbox
        input_file = input_files[data_idx]
        img_origin = Image.open(join(img_dir, input_file)).convert('RGB')
        bboxes_true = data_dict[input_file][1]
        bboxes_true = util.bboxes_resize(img_origin, bboxes_true, size=224)

        # get ours bbox
        bbox_propose = bboxes_dict[input_file]

        # get iou
        iou_propose = []
        for bbox_true in bboxes_true:
            iou_propose.append(util.get_iou(bbox_true, bbox_propose))
        iou_propose = max(np.array(iou_propose) >= 0.5).astype(np.int)
        gtknown_acc_locs.append(iou_propose)
        
    top1_acc_locs = np.logical_and(top1_acc_clss, gtknown_acc_locs)
    top5_acc_locs = np.logical_and(top5_acc_clss, gtknown_acc_locs)
    
    top1_acc_cls += top1_acc_clss.sum()
    top5_acc_cls += top5_acc_clss.sum()
    gtknown_acc_loc += np.array(gtknown_acc_locs).sum()
    top1_acc_loc += top1_acc_locs.sum()
    top5_acc_loc += top5_acc_locs.sum()
    
#     if idx == 999:
#         break

top1_acc_cls /= count
top5_acc_cls /= count
gtknown_acc_loc /= count
top1_acc_loc /= count
top5_acc_loc /= count
print('Classification & Localization')
print(f'===> Top1-Loc: {top1_acc_loc}, Top1-Acc: {top1_acc_cls}, GTKnown: {gtknown_acc_loc}')
print(f'===> Top5-Loc: {top5_acc_loc}, Top5-Acc: {top5_acc_cls}')


#### vgg16_bn 
vanilla ===> Top1-Loc: 0.44906, Top1-Acc: 0.7336, GTKnown: 0.57684, Top5-Loc: 0.54378, Top5-Acc: 0.91516 <br>
ours ===> Top1-Loc: 0.47832, Top1-Acc: 0.7336, GTKnown: 0.61506, Top5-Loc: 0.5803, Top5-Acc: 0.91516

In [ ]:
join('aa','bb','','cc')

## Save the plot

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]='2'

from os.path import join
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from tqdm import tqdm_notebook

from cam import CAM
import util

In [ ]:
map = CAM()

class_dict = dataloader.get_class_dict('/data2/imagenet')[0]
data_dict = map.valid_dataset.data_dict
input_files = map.valid_dataset.img_files
img_dir = map.valid_dataset.img_dir

In [ ]:
def save_result_plt(data_idx, th1=0.2, th2=10, mc=15):
    # get true bbox
    input_file = input_files[data_idx]
    img_origin = Image.open(join(img_dir, input_file)).convert('RGB')
    bboxes_true = data_dict[input_file][1]
    bboxes_true = util.bboxes_resize(img_origin, bboxes_true, size=224)
    
    # get input, target, and topk
    input, target = map.get_item(data_idx)
    target = target.cpu().item()

    topk_idxs = map.topk(input)
    top1_correct = target in topk_idxs[:1]
    top5_correct = target in topk_idxs[:5]
    
    att_idx = target
    
    # origin
    img, heatmap_origin, boolmap, boolmap_biggest, \
    bbox_pred = map.get_values(data_idx,att_idx, th1=0.2, phase='test')
    
    # propose
    _, heatmap_mean, heatmap_std, boolmap_propose, boolmap_biggest_propose, \
    bbox_propose = map.get_values(data_idx, att_idx, th1, th2, mc, phase='train')
    heatmap_std_max = heatmap_std.max()
    
    # save the plot
    fig, ax = plt.subplots(2,6,figsize=(21,7))

    ax[0,0].imshow(img)
    ax[0,0].set_title('origin')
    ax[0,0].axis('off')

    ax[0,1].imshow(heatmap_origin)
    ax[0,1].set_title('heatmap')
    ax[0,1].axis('off')

    ax[0,2].imshow(img)
    ax[0,2].imshow(heatmap_origin, alpha=0.5, cmap='jet')
    ax[0,2].set_title(f'CAM "{class_dict[target]}"\n Top-1: {top1_correct}, Top-5: {top5_correct}')
    ax[0,2].axis('off')

    ax[0,3].imshow(Image.fromarray((boolmap*255).astype(np.uint8)), cmap='gray')
    ax[0,3].set_title('boolean map')
    ax[0,3].axis('off')

    ax[0,4].imshow(Image.fromarray((boolmap_biggest*255).astype(np.uint8)), cmap='gray')
    ax[0,4].set_title('biggest boolean map')
    ax[0,4].axis('off')

    ax[0,5].imshow(img)
    for bbox_true in bboxes_true:
        rect_true = patches.Rectangle((bbox_true[0],bbox_true[1]),bbox_true[2],bbox_true[3],
                                      linewidth=2,edgecolor='g',facecolor='none')
        ax[0,5].add_patch(rect_true)
    rect_pred = patches.Rectangle((bbox_pred[0],bbox_pred[1]),bbox_pred[2],bbox_pred[3],
                                  linewidth=2,edgecolor='r',facecolor='none')
    ax[0,5].add_patch(rect_pred)
    ax[0,5].set_title('bounding box')
    ax[0,5].axis('off')


    ax[1,0].imshow(heatmap_mean, cmap='gray')
    ax[1,0].set_title('heatmap_mean')
    ax[1,0].axis('off')

    ax[1,1].imshow(heatmap_std, cmap='gray')
    ax[1,1].set_title(f'heatmap_std\n max value: {heatmap_std_max:.01f}')
    ax[1,1].axis('off')

    im1 = ax[1,2].imshow(heatmap_mean, cmap='Reds', label='mean')
    im2 = ax[1,2].imshow(heatmap_std, cmap='Blues', label='std', alpha=0.5)
    ax[1,2].set_title('overlap')
    ax[1,2].axis('off')
    patch = [patches.Patch(color=im1.cmap(150), label='mean'), 
             patches.Patch(color=im2.cmap(150), label='std')]
    ax[1,2].legend(handles=patch, loc='best')

    ax[1,3].imshow(Image.fromarray((boolmap_propose*255).astype(np.uint8)), cmap='gray')
    ax[1,3].set_title('boolean map')
    ax[1,3].axis('off')

    ax[1,4].imshow(Image.fromarray((boolmap_biggest_propose*255).astype(np.uint8)), cmap='gray')
    ax[1,4].set_title('biggest boolean map')
    ax[1,4].axis('off')

    ax[1,5].imshow(img)
    for bbox_true in bboxes_true:
        rect_true = patches.Rectangle((bbox_true[0],bbox_true[1]),bbox_true[2],bbox_true[3],
                                      linewidth=2,edgecolor='g',facecolor='none')
        ax[1,5].add_patch(rect_true)
    rect_pred = patches.Rectangle((bbox_propose[0],bbox_propose[1]),bbox_propose[2],bbox_propose[3],
                                  linewidth=2,edgecolor='r',facecolor='none')
    ax[1,5].add_patch(rect_pred)
    ax[1,5].set_title('bounding box')
    ax[1,5].axis('off')

    plt.show()
#     plt.savefig(join('save',f'{data_idx:05d}.png'))
#     plt.close(fig)

In [ ]:
# for data_idx in tqdm_notebook(range(1000)):
#     save_result_plt(data_idx, th1=0.4, th2=0.4, mc=30)

In [ ]:
save_result_plt(129,th1=0.4, th2=0.4, mc=50) # good hyperparam... maybe! th1=0.4, th2=0.4, mc=50 